In [ ]:
#BOLSHOI
#Performance ids are unknown to us, so we'll iterate over 2000 possible ids, if the url works it means this is a valid
#entry and we can try to parse it.
#Look for relevant performance dates, as this value will reference a hidden table that contains the cast for that day

from bs4 import BeautifulSoup
import requests

name = "Russia/Bolshoi.csv"
csv = open(name, "w") 
columnTitleRow = "opera_title\t opera_description\t date\t role\t artist\t url\n"
csv.write(columnTitleRow)

for perf in list(range(1, 2)): #up to 2000
    url = "https://www.bolshoi.ru/en/performances/" + str(perf) + "/roles/#all"
    r  = requests.get(url)
    soup = BeautifulSoup(r.text, "html5lib")
    try: 
        opera_title = soup.select('.navigation_title')[0].text
    except IndexError:
        opera_title = ''
    try:     
        opera_description = soup.select(".perf_desc_title")[0].text
    except IndexError:
        opera_description = ''
        
    for perf_date in soup.select(".performance_main_content__dates_all .cast_selector"):
        date = perf_date.get('href')[1:]
        cast = ".cast_" + str(date)
        for c in soup.select(cast + ' tr'):
            role = c.select('.performance_main_content__cast_role_name')[0].text
            artist = c.select('.performance_main_content__cast_role_person')[0].text.splitlines()
            artist = [ x.strip() for x in artist ]
            artist[:] = [value for value in artist if len(value)!=0]
            artist = ",".join(artist)
#             print(opera_title + '-' + opera_description + '-' + date + '-' + role + '-' + artist )
            row = opera_title + "\t" + opera_description + "\t" + date + "\t" + role + "\t" + artist + "\t" + url + "\n"
            csv.write(row)
            


In [ ]:
#MARIINSKY
#Multiple navigation layers, first season, which by default opens in its first month (usually september - not a url)
#then there's a second level month selection, once month is selected a full breakdown opens for each day.
#there can be multiple entries on the same day as they have matinees and other theaters.
#not all operas have cast
#cast table not really a table, just a div with some links, so have to construct pairs out of this.

from bs4 import BeautifulSoup
import requests

#We'll need this function as we will have a mix of string and links and garbage
def process_marrinsky(row):
    if row.name == None:
        return row.split('–')
    else: 
        return row.text    
    
def flatten(lst):
    for el in lst:
        if isinstance(el, list):
            yield from el
        else:
            yield el     
            
def clean_cast(cast):
    cast = [row for row in cast if row.name != 'br']
    cast = [process_marrinsky(row) for row in cast]
    cast = list(flatten(cast))
    cast = [row.strip() for row in cast]
    cast = [row for row in cast if len(row) > 0]  
    return cast

name = "Russia/Mariinksy.csv"
csv = open(name, "w") 
columnTitleRow = "opera_title\t opera_description\t date\t role\t artist\t url\n"
csv.write(columnTitleRow)
base_url = "https://www.mariinsky.ru"

for season in list(range(226, 227)): #up to 2000
    url = base_url + "/en/playbill/archive?season=" + str(season)
    r  = requests.get(url)
    season_main = BeautifulSoup(r.text, "html5lib")
    ################################################################################################
    #First month of the season is not a URL so we'll parse before we deal with the subsequent pages.
    ################################################################################################
    for day in season_main.select('div#afisha .row'):
        date = day.select("time")[0]['datetime']
        opera_title = day.select('.spec_name')[0].text
        opera_description = day.select('.descr')[0].text
        if 'opera' in opera_description:
            opera_url = base_url + day.select('.spec_name a')[0].get('href')
            r_opera  = requests.get(opera_url)
            opera_main = BeautifulSoup(r_opera.text, "html5lib")
            if len(opera_main.select('.sostav.inf_block')) > 0:   
                cast_table = clean_cast(opera_main.select('.sostav.inf_block')[0].find_all('div')[1:][0].contents)              
                artists = []
                roles = []
                for item in cast_table:
                    if cast_table.index(item) % 2 != 0:
                        artists.append(item)
                    else:
                        roles.append(item)

                roles_artists = list(zip(roles, artists))    

                for cast_row in roles_artists:
                    row = opera_title + "\t" + opera_description + "\t" + date + "\t" + cast_row[0] + "\t" + cast_row[1] + "\t" + opera_url + "\n"
                    csv.write(row)
            else:
                row = opera_title + "\t" + opera_description + "\t" + date + "\t" + 'No info' + "\t" + 'No info' + "\t" + opera_url + "\n"
                csv.write(row)       
    ################################################################################################
    for month in season_main.select(".playbill_archive_months a"):
        month_name = month.text
        month_url = base_url + month.get('href')
        r_month  = requests.get(month_url)
        month_main = BeautifulSoup(r_month.text, "html5lib")
        for day in month_main.select('div#afisha .row'):
            date = day.select("time")[0]['datetime']
            opera_title = day.select('.spec_name')[0].text
            opera_description = day.select('.descr')[0].text
            if 'opera' in opera_description: #Other ballets and concerts have a different structure and are of no interest.            
                opera_url = base_url + day.select('.spec_name a')[0].get('href')
                r_opera  = requests.get(opera_url)
                opera_main = BeautifulSoup(r_opera.text, "html5lib")
                if len(opera_main.select('.sostav.inf_block')) > 0:   
                    cast_table = clean_cast(opera_main.select('.sostav.inf_block')[0].find_all('div')[1:][0].contents)              
                    artists = []
                    roles = []
                    for item in cast_table:
                        if cast_table.index(item) % 2 != 0:
                            artists.append(item)
                        else:
                            roles.append(item)

                    roles_artists = list(zip(roles, artists))    

                    for cast_row in roles_artists:
                        row = opera_title + "\t" + opera_description + "\t" + date + "\t" + cast_row[0] + "\t" + cast_row[1] + "\t" + opera_url + "\n"
                        csv.write(row)
                else:
                    row = opera_title + "\t" + opera_description + "\t" + date + "\t" + 'No info' + "\t" + 'No info' + "\t" + opera_url + "\n"
                    csv.write(row)                            

In [345]:
#Ekaterinburg Opera
#Performances since 2009, no cast data
#TODO for some reason writing to csv ends in June 2017... look into this.
from bs4 import BeautifulSoup
import requests

name = "Russia/Ekaterinburg.csv"
csv = open(name, "w") 
columnTitleRow = "opera_title\t opera_description\t date\t role\t artist\t url\n"
csv.write(columnTitleRow)

years = list(range(2009,2019))
months = list(range(1,13))

for year in years:
    for month in months:
        url = 'http://www.uralopera.ru/en/playbill.php/?param=month&year=' + str(year) + '&month=' + str(month)
        r  = requests.get(url)
        opera_month = BeautifulSoup(r.text, "html5lib")
        opera_rows = opera_month.select('p tr')
        for opera_row in opera_rows:
            opera_row_items = opera_row.find_all('td', {'align': 'center'})
            if len(opera_row_items) == 3:                 
                if 'opera' in opera_row_items[1].text.lower():
                    date = opera_row_items[0].text[0:-11]
                    opera_title = opera_row_items[1].select('.big_black')[0].text
                    row = opera_title + "\t" + 'No info' + "\t" + date + "\t" + 'No info' + "\t" + 'No info' + "\t" + url + "\n"
                    csv.write(row)   

In [ ]:
#Novat Opera
#Performances since 2015

from bs4 import BeautifulSoup
import requests

name = "Russia/Novosibirsk.csv"
csv = open(name, "w") 
columnTitleRow = "opera_title\t opera_description\t date\t role\t artist\t url\n"
csv.write(columnTitleRow)

years = list(range(2015,2018))
months = list(range(1,13))
start_months = list(range(11,13))

for year in years:
    if year == 2015:
        for month in start_months:
            url = 'https://novat.nsk.ru/en/afisha/performances/' + str(year) + '/' + str(month) + '/'
            r  = requests.get(url)
            opera_month = BeautifulSoup(r.text, "html5lib")
            opera_rows = opera_month.select('.data-item__content .poster-item__title')
            for opera_row in opera_rows:     
                opera_url = 'https://novat.nsk.ru' + opera_row.get('href')
                r_opera  = requests.get(opera_url)
                opera_page = BeautifulSoup(r_opera.text, "html5lib")
                date = str(year) + '-' + str(month) + '-' + opera_page.select('.date_calendar__date.c-date_calendar')[0].text.strip().split(' ')[0]                
                opera_title = opera_row.text.strip()
                try:
                    opera_descriptions = opera_page.select('.actor-header__main-info p')
                    opera_description = []
                    for item in opera_descriptions:
                        opera_description.append(item.text)
                    opera_description = '-'.join(opera_description)
                except:
                    opera_description = ''                    
                if 'opera' in opera_description:    
                    for cast_row in opera_page.select('.artist.artist--master'):
                        artist = cast_row.select('.artist__name')[0].text.strip()
                        role = cast_row.select('.artist__position')[0].text.strip()
                        row = opera_title + "\t" + opera_description + "\t" + date + "\t" + role + "\t" + artist + "\t" + opera_url + "\n"
                        csv.write(row)   
    else:
        for month in months:
            url = 'https://novat.nsk.ru/en/afisha/performances/' + str(year) + '/' + str(month) + '/'
            r  = requests.get(url)
            opera_month = BeautifulSoup(r.text, "html5lib")
            opera_rows = opera_month.select('.data-item__content .poster-item__title')
            for opera_row in opera_rows:
                opera_url = 'https://novat.nsk.ru' + opera_row.get('href')
                r_opera  = requests.get(opera_url)
                opera_page = BeautifulSoup(r_opera.text, "html5lib")
                date = str(year) + '-' + str(month) + '-' + opera_page.select('.date_calendar__date.c-date_calendar')[0].text.strip().split(' ')[0]                
                opera_title = opera_row.text.strip()
                try:
                    opera_descriptios = opera_page.select('.actor-header__main-info p')
                    opera_description = []
                    for item in opera_descriptions:
                        opera_description.append(item.text)
                    opera_description = '-'.join(opera_description)
                except:
                    opera_description = ''
                if 'opera' in opera_description:    
                    for cast_row in opera_page.select('.artist.artist--master'):
                        artist = cast_row.select('.artist__name')[0].text.strip()
                        role = cast_row.select('.artist__position')[0].text.strip()
                        row = opera_title + "\t" + opera_description + "\t" + date + "\t" + role + "\t" + artist + "\t" + opera_url + "\n"
                        csv.write(row)          


In [72]:
#Mihailovsky Theatre St. Petersburg
#Performances since 2013
from bs4 import BeautifulSoup
import requests

def process_mihailovsky_desc(row):
    if row.name == None:
        return row.split('\n')
    else: 
        return row.text

def clean_cast(cast):
    cast = [row for row in cast if row.name != 'br']
    cast = [process_marrinsky(row) for row in cast]
    cast = list(flatten(cast))
    cast = [row.strip() for row in cast]
    cast = [row for row in cast if len(row) > 0]  
    cast = [row.replace('\xa0', ' ') for row in cast]      
    cast = [row.replace(' —', ' ') for row in cast]      
    return cast

name = "Russia/Mihailovsky.csv"
csv = open(name, "w") 
columnTitleRow = "opera_title\t opera_description\t date\t role\t artist\t url\n"
csv.write(columnTitleRow)

years = list(range(2013,2019))
months = list(range(1,13))

for year in years:
    for month in months:
        url = 'https://www.mikhailovsky.ru/en/afisha/performances/' + str(year) + '/' + str(month) + '/'
        r  = requests.get(url)
        opera_month = BeautifulSoup(r.text, "html5lib")
        for performance in opera_month.select('h2 a'):
            opera_title = performance.text.strip()
            opera_url = 'https://www.mikhailovsky.ru' + performance.get('href')
            r_opera = requests.get(opera_url)
            opera_page = BeautifulSoup(r_opera.text, "html5lib")
            date = str(year) + '-' + str(month) + '-' + opera_page.select('.date')[0].find_all("strong")[0].text
            opera_description = opera_page.select('.detail')[0].contents
            opera_description = [process_mihailovsky_desc(row) for row in opera_description]
            opera_description = list(flatten(opera_description))    
            opera_description = [row.replace('\t', '') for row in opera_description]
            opera_description = [row.replace('\n', ' ') for row in opera_description]
            opera_description = [row.strip() for row in opera_description]
            opera_description = [row for row in opera_description if len(row) > 0]  
            opera_description = '-'.join(opera_description)
            try: 
                cast = opera_page.select('dd:nth-of-type(1)')[0].contents
                cast = clean_cast(cast)
                artists = []
                roles = []
                for item in cast:
                    if cast.index(item) % 2 != 0:
                        artists.append(item)
                    else:
                        roles.append(item)

                roles_artists = list(zip(roles, artists))    
            except:
                roles_artists = []
            for cast_row in roles_artists:
                row = opera_title + "\t" + opera_description + "\t" + date + "\t" + cast_row[0] + "\t" + cast_row[1] + "\t" + opera_url + "\n"
                csv.write(row)

KeyboardInterrupt: 

In [59]:
# St Petersburg Chamber Opera
#Performances since 2015
from bs4 import BeautifulSoup
import requests


name = "Russia/StPetersburgChamber.csv"
csv = open(name, "w") 
columnTitleRow = "opera_title\t opera_description\t date\t role\t artist\t url\n"
csv.write(columnTitleRow)

years = list(range(2015,2019))
months = list(range(1,13))
months = ["%02d" % n for n in months]
first_months = list(range(3,13))
first_months = ["%02d" % n for n in first_months]

output = ["%02d" % n for n in output]

for year in years:
    if year == '2015':
        for month in first_months:
            url = 'http://www.spbopera.ru/en/afisha/' + str(year) + '-' + str(month)
            r  = requests.get(url)
            opera_month = BeautifulSoup(r.text, "html5lib")
            

    else:
        
        for month in months:
            url = 'https://www.mikhailovsky.ru/en/afisha/performances/' + str(year) + '/' + str(month) + '/'
            r  = requests.get(url)
            opera_month = BeautifulSoup(r.text, "html5lib")
            for performance in opera_month.select('h2 a'):
                opera_title = performance.text.strip()
                opera_url = 'https://www.mikhailovsky.ru' + performance.get('href')
                r_opera = requests.get(opera_url)
                opera_page = BeautifulSoup(r_opera.text, "html5lib")
                date = str(year) + '-' + str(month) + '-' + opera_page.select('.date')[0].find_all("strong")[0].text
                opera_description = opera_page.select('.detail')[0].contents
                opera_description = [process_mihailovsky_desc(row) for row in opera_description]
                opera_description = list(flatten(opera_description))    
                opera_description = [row.replace('\t', '') for row in opera_description]
                opera_description = [row.replace('\n', ' ') for row in opera_description]
                opera_description = [row.strip() for row in opera_description]
                opera_description = [row for row in opera_description if len(row) > 0]  
                opera_description = '-'.join(opera_description)
                try: 
                    cast = opera_page.select('dd:nth-of-type(1)')[0].contents
                    cast = clean_cast(cast)
                    artists = []
                    roles = []
                    for item in cast:
                        if cast.index(item) % 2 != 0:
                            artists.append(item)
                        else:
                            roles.append(item)

                    roles_artists = list(zip(roles, artists))    
                except:
                    roles_artists = []
                for cast_row in roles_artists:
                    row = opera_title + "\t" + opera_description + "\t" + date + "\t" + cast_row[0] + "\t" + cast_row[1] + "\t" + opera_url + "\n"
                    csv.write(row)

In [79]:
# St Petersburg Chamber Opera
#Performances since 2015
from bs4 import BeautifulSoup
import requests

url = 'http://www.spbopera.ru/en/afisha/2015-04'
r  = requests.get(url)
opera_month = BeautifulSoup(r.text, "html5lib")
opera_month

ConnectionError: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response',))

In [75]:
# months = list(range(1,13))
# months = ["%02d" % n for n in months]
# months